In [ ]:
import numpy as np
import pandas as pd
import os
import re
import gc
from tqdm import tqdm

In [ ]:
train_df=pd.read_csv("/kaggle/input/novozymes-enzyme-stability-prediction/train.csv")#len(train_df)=31390
#train_df["len"]=train_df["protein_sequence"].str.len()
train_updates_20220929_df=pd.read_csv("/kaggle/input/novozymes-enzyme-stability-prediction/train_updates_20220929.csv")#len(train_updates_20220929_df)=2409
train_df2=train_df[~train_df["seq_id"].isin(train_updates_20220929_df["seq_id"]) & ~train_df["pH"].isna()].reset_index()
train_df2.head()

In [ ]:
del train_df, train_updates_20220929_df
_= gc.collect()

In [ ]:
import pickle

with open('../input/train-seq-unirep/train_seq_unirep(h_avg).pkl', 'rb') as f:
    X= pickle.load(f)

In [ ]:
y=train_df2["tm"]

In [ ]:
import scipy
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC, SVR
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression, SGDRegressor, Ridge,Lasso
from sklearn.pipeline import Pipeline

In [ ]:
from sklearn.metrics import make_scorer

def spearman(y_train, y_pred):
    return scipy.stats.spearmanr(y_train, y_pred)#[0]

#https://www.kaggle.com/code/ryanluoli2/regression-and-tree-baselines-lb-0-80
#scorer = make_scorer(spearman, greater_is_better=True)

In [ ]:
Xtrain, Xtest, ytrain, ytest=train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
lasso_reg = linear_model.Lasso(alpha=0.1)
lasso_reg.fit(Xtrain,ytrain)

ypred=lasso_reg.predict(Xtest)

In [ ]:
ridge_reg = linear_model.Ridge(alpha=0.1)
ridge_reg.fit(Xtrain,ytrain)

print(ridge_reg.coef_)
print(ridge_reg.intercept_)

In [ ]:
ypred=ridge_reg.predict(Xtest)

In [ ]:
print(spearman(ytest,ypred))

In [ ]:
with open('../input/test-seq-unirep/test_seq_unirep(h_avg).pkl', 'rb') as f:
    submission_X= pickle.load(f)

In [ ]:
ridge_reg.fit(X,y)
submission_y=ridge_reg.predict(submission_X)

In [ ]:
submission_df = pd.read_csv('../input/novozymes-enzyme-stability-prediction/sample_submission.csv')
submission_df.head()

In [ ]:
submission=submission_df.copy()
submission['tm'] = submission_y
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.head()